<a href="https://colab.research.google.com/github/KwinLook/ADEX-Project/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
import torchvision
from torchvision import datasets, transforms
from torch import nn,optim
from time import time

In [ ]:
width = 150
size=(width,width)
tranform = transforms.Compose([transforms.Resize(size),
                              transforms.Grayscale(num_output_channels=1),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5),(0.5))])

In [ ]:
trainset = torchvision.datasets.ImageFolder(root="train/", transform=tranform)
print(trainset)

In [ ]:
validationset = torchvision.datasets.ImageFolder(root="validation/", transform=tranform)
print(validationset)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
print(trainloader)

In [ ]:
validationloader = torch.utils.data.DataLoader(validationset, batch_size=4, shuffle=True)
print(validationloader)

In [ ]:
images, labels = next(iter(trainloader))

In [ ]:
print(images.shape)
print(labels.shape)
print(labels)

In [ ]:
#Create ANN Model
input_size = width*width*1
hidden_sizes = [1890,1350,810,270,135,60]
output_size = 3

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]), nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]), nn.ReLU(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]), nn.ReLU(),
                      nn.Linear(hidden_sizes[2], hidden_sizes[3]), nn.ReLU(),
                      nn.Linear(hidden_sizes[3], hidden_sizes[4]), nn.ReLU(),
                      nn.Linear(hidden_sizes[4], hidden_sizes[5]), nn.ReLU(),
                      nn.Linear(hidden_sizes[5], output_size), nn.LogSoftmax(dim=1))

print(model)

In [ ]:
#Train Model
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.09)
time0 = time()
epochs = 100
for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    x = images.view(images.shape[0], -1)
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print('Epoch {} - Train loss {}'.format(e+1, running_loss/len(trainloader)))
print('\nTraining Time (in minutes) - {}'.format((time()-time0)/60))

In [ ]:
images_validation,labels_validation=next(iter(validationloader))
print(images_validation.shape)
print(labels_validation)

In [ ]:
x_validation = images_validation[1].view(1,width*width)

with torch.no_grad():
  y = model(x_validation)

print(torch.exp(y))

plt.imshow(images_validation[1][0], cmap='gray')
plt.show()
print('ANSWER:' ,labels[1])


In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0

for images_validation, labels_validation in validationloader:
  for i in range(len(labels_validation)):
    x = images_validation[i].view(1,150*150)

    with torch.no_grad():
        log_y = model(x)
    y = torch.exp(log_y)
    prob = list(y.numpy()[0])
    pred_output = prob.index(max(prob))

    answer = labels_validation.numpy()[i]
    if answer == pred_output and answer == 0:
          true_negatives += 1
    elif answer == pred_output and answer == 1:
          true_positives += 1
    elif answer != pred_output and answer == 0:
          false_positives += 1
    elif answer != pred_output and answer == 1:
          false_negatives += 1

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1 = 2 * (precision * recall) / (precision + recall)

print('Number of Images validated:', true_positives + false_positives + false_negatives + true_negatives)
print('Model Accuracy:', (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives))
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

**Prediction With Unseen Data**

In [ ]:
predictset = torchvision.datasets.ImageFolder(root="predict-750-images/", transform=tranform)
print(predictset)

In [ ]:
predictloader = torch.utils.data.DataLoader(predictset, batch_size=4, shuffle=True)
print(predictloader)

In [ ]:
images_predict,labels_predict=next(iter(predictloader))
print(images_predict.shape)
print(labels_predict)

In [ ]:
x_predict = images_predict[1].view(1,width*width)

with torch.no_grad():
  y = model(x_predict)

print(torch.exp(y))

plt.imshow(images_predict[1][0], cmap='gray')
plt.show()
print('ANSWER:' ,labels[1])


In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0

for images_predict, labels_predict in predictloader:
  for i in range(len(labels_predict)):
    x = images_predict[i].view(1,150*150)

    with torch.no_grad():
        log_y = model(x)
    y = torch.exp(log_y)
    prob = list(y.numpy()[0])
    pred_output = prob.index(max(prob))

    answer = labels_predict.numpy()[i]
    if answer == pred_output and answer == 0:
          true_negatives += 1
    elif answer == pred_output and answer == 1:
          true_positives += 1
    elif answer != pred_output and answer == 0:
          false_positives += 1
    elif answer != pred_output and answer == 1:
          false_negatives += 1
          
precision = true_positives / (true_positives + false_positives)
if (true_positives + false_negatives) == 0:
    recall = 0
else:
    recall = true_positives / (true_positives + false_negatives)
if (precision + recall) == 0:
    f1 = 0
else:
    f1 = 2 * (precision * recall) / (precision + recall)

print('Number of Images validated:', true_positives + false_positives + false_negatives + true_negatives)
print('Model Accuracy:', (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives))
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)